<a href="https://colab.research.google.com/github/skrofman/Applied-Machine-Learning/blob/master/DATA_310_Lecture_21_Spring_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
import xgboost as xgb 

In [ ]:
dat = load_boston()
df = pd.DataFrame(data=dat.data,columns=dat.feature_names)

In [ ]:
X = dat.data
y = dat.target

In [ ]:
X.shape

(506, 13)

## Definition: Root Mean Squared Error

$$\mbox{RMSE}:=\sqrt{\frac{1}{n}\sum_{i=1}^{p}(y_i-\hat{y_i})^2}$$

(in "loose" terms this is sometimes called the empirical norm)

In [ ]:
def RMSE(y,yhat):
  return np.sqrt(MSE(y,yhat))

In [ ]:
def MAEf(y,yhat):
  n=len(y)
  return np.sum(np.abs(y-yhat))/n

In [ ]:
def DoKFold(X,y,model,k):
  PE1 = []
  PE2 = []
  kf = KFold(n_splits=k,shuffle=True,random_state=1234)
  for idxtrain,idxtest in kf.split(X):
    Xtrain = X[idxtrain,:]
    Xtest  = X[idxtest,:]
    ytrain = y[idxtrain]
    ytest  = y[idxtest]
    model.fit(Xtrain,ytrain)
    yhat = model.predict(Xtest)
    PE1.append(MAEf(ytest,yhat))
    PE2.append(RMSE(ytest,yhat))
  return 1000*np.mean(PE1), 1000*np.mean(PE2)

### Querstion: Do we need scaling for Random Forest?
 Answer: No.

In [ ]:
model = RandomForestRegressor(n_estimators=500,max_depth=10,random_state=1234)

In [ ]:
DoKFold(X,y,model,10)

(2273.5560051999537, 3394.701250044423)

In [ ]:
model = xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=500,reg_lambda=1,alpha=1,gamma=1,max_depth=10,random_state=1234)

In [ ]:
DoKFold(X,y,model,10)

(2092.731559730978, 3176.011342160705)

In [ ]:
# imports for creating a Neural Networks (Deep Learning)
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import r2_score
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [ ]:
# Create a Neural Network model
model = Sequential()
model.add(Dense(128, activation="relu", input_dim=13))
model.add(Dense(32, activation="relu"))
model.add(Dense(8, activation="relu"))
# Since the regression is performed, a Dense layer containing a single neuron with a linear activation function.
# Typically ReLu-based activation are used but since it is performed regression, it is needed a linear activation.
model.add(Dense(1, activation="linear"))

# Compile model: The model is initialized with the Adam optimizer and then it is compiled.
model.compile(loss='mean_squared_error', optimizer=Adam(lr=1e-3, decay=1e-3 / 200))

# Patient early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1000)


In [ ]:
mae_nn = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
for idxtrain, idxtest in kf.split(X):
  X_train = X[idxtrain,:]
  y_train = y[idxtrain]
  X_test  = X[idxtest,:]
  y_test = y[idxtest]
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
  model.fit(X_train,y_train,validation_split=0.3, epochs=1000, batch_size=100, verbose=0, callbacks=[es])
  yhat_nn = model.predict(X_test)
  mae_nn.append(MAE(y_test, yhat_nn))
print("Validated MAE Neural Network Regression = ${:,.2f}".format(1000*np.mean(mae_nn)))

Epoch 00466: early stopping
Epoch 00254: early stopping
Epoch 00272: early stopping
Epoch 00206: early stopping
Epoch 00246: early stopping
Epoch 00204: early stopping
Epoch 00393: early stopping
Epoch 00296: early stopping
Epoch 00638: early stopping
Validated MAE Neural Network Regression = $4,127.29
